In [42]:
import os
import dotenv
import argparse
from typing import List, Dict, Any, Optional, Tuple
import psycopg2
from psycopg2.extras import RealDictCursor
import sys
import os
from vector_processor import VectorProcessor
from html_cleaner import HTMLCleaner

dotenv.load_dotenv(override=True)
db_name = os.getenv("DB_NAME", "VectorDB01")
db_user = os.getenv("DB_USER", "postgres")
db_password = os.getenv("DB_PASSWORD", "Admin")
db_host = os.getenv("DB_HOST", "localhost")
db_port = os.getenv("DB_PORT", "5432")


class VectorDBQuerier:
    def __init__(self, vector_processor: VectorProcessor):
        """Initialize the vector database querier.
        
        Args:
            vector_processor: Instance of VectorProcessor for creating embeddings
        """
        
        self.vector_processor = vector_processor
        self.connection = self._get_connection()
        print(f"Connecting to the database {db_name}")
        
    def _get_connection(self):
        """Establish database connection."""
        connection_params = {
            "host": db_host,
            "port": db_port,
            "database": db_name,
            "user": db_user,
            "password": db_password,
            "cursor_factory": RealDictCursor
        }
        return psycopg2.connect(**connection_params)
    
    def semantic_search(self, query: str, limit: int = 5, similarity_threshold: float = 0.5) -> List[Dict[str, Any]]:
        """Search for semantically similar content based on vector similarity.
        
        Args:
            query: The search query text
            limit: Maximum number of results to return
            similarity_threshold: Minimum similarity score (0-1) to include in results
            
        Returns:
            List of matching segments with their metadata and similarity scores
        """
        try:
            query_embedding = self.vector_processor.create_embedding(query)
            
            cursor = self.connection.cursor()
            cursor.execute("""
        SELECT 
            ps.id AS segment_id,
            ps.page_id,
            ps.page_segment,
            cp.url,
            1 - (ps.embedding <=> %s::vector) AS similarity
        FROM 
            crawldb.page_segment ps
        JOIN 
            crawldb.cleaned_page cp ON ps.page_id = cp.id
        WHERE 
            1 - (ps.embedding <=> %s::vector) > %s
        ORDER BY 
            similarity DESC
        LIMIT %s;
    """, (query_embedding, query_embedding, similarity_threshold, limit))

            
            results = cursor.fetchall()
            cursor.close()
            
            return results
            
        except Exception as e:
            print(f"Error in semantic search: {str(e)}")
            return []
    
    def keyword_and_semantic_search(self, query: str, limit: int = 5) -> List[Dict[str, Any]]:
        """Hybrid search combining keyword matching and semantic similarity."""
        try:
            query_embedding = self.vector_processor.create_embedding(query)
            cursor = self.connection.cursor()

            cursor.execute("""
                SELECT 
                    ps.id AS segment_id,
                    ps.page_id,
                    ps.page_segment,
                    cp.url,
                    ts_rank_cd(to_tsvector('english', ps.page_segment), plainto_tsquery('english', %s)) AS text_rank,
                    1 - (ps.embedding <=> %s::vector) AS vector_similarity,
                    (ts_rank_cd(to_tsvector('english', ps.page_segment), plainto_tsquery('english', %s)) * 0.4 + 
                    (1 - (ps.embedding <=> %s::vector)) * 0.6) AS combined_score
                FROM 
                    crawldb.page_segment ps
                JOIN 
                    crawldb.cleaned_page cp ON ps.page_id = cp.id
                WHERE 
                    to_tsvector('english', ps.page_segment) @@ plainto_tsquery('english', %s)
                    OR 1 - (ps.embedding <=> %s::vector) > 0.6
                ORDER BY 
                    combined_score DESC
                LIMIT %s;
            """, (query, query_embedding, query, query_embedding, query, query_embedding, limit))

            results = cursor.fetchall()
            cursor.close()
            return results
            
        except Exception as e:
            print(f"Error in hybrid search: {str(e)}")
            return []
    
    def url_content_search(self, url_pattern: str, query: str = None, limit: int = 5) -> List[Dict[str, Any]]:
        """Search for content within URLs matching a pattern, optionally filtered by query."""
        try:
            cursor = self.connection.cursor()

            if query:
                query_embedding = self.vector_processor.create_embedding(query)

                cursor.execute("""
                    SELECT 
                        ps.id AS segment_id,
                        ps.page_id,
                        ps.page_segment,
                        cp.url,
                        1 - (ps.embedding <=> %s::vector) AS similarity
                    FROM 
                        crawldb.page_segment ps
                    JOIN 
                        crawldb.cleaned_page cp ON ps.page_id = cp.id
                    WHERE 
                        cp.url LIKE %s
                    ORDER BY 
                        similarity DESC
                    LIMIT %s;
                """, (query_embedding, f"%{url_pattern}%", limit))
            else:
                cursor.execute("""
                    SELECT 
                        ps.id AS segment_id,
                        ps.page_id,
                        ps.page_segment,
                        cp.url
                    FROM 
                        crawldb.page_segment ps
                    JOIN 
                        crawldb.cleaned_page cp ON ps.page_id = cp.id
                    WHERE 
                        cp.url LIKE %s
                    LIMIT %s;
                """, (f"%{url_pattern}%", limit))

            results = cursor.fetchall()
            cursor.close()

            return results

        except Exception as e:
            print(f"Error in URL search: {str(e)}")
            return []

    
    def get_page_content(self, page_id: int) -> Tuple[str, str]:
        """Retrieve the full content of a specific page.
        
        Args:
            page_id: ID of the page to retrieve
            
        Returns:
            Tuple of (URL, full text content)
        """
        try:
            cursor = self.connection.cursor()
            
            cursor.execute("""
                SELECT url, plain_text
                FROM crawldb.cleaned_page
                WHERE id = %s;
            """, (page_id,))
            
            result = cursor.fetchone()
            cursor.close()
            
            if result:
                return result['url'], result['plain_text']
            else:
                return None, None
                
        except Exception as e:
            print(f"Error retrieving page content: {str(e)}")
            return None, None
    
    def close(self):
        """Close the database connection."""
        if self.connection:
            self.connection.close()


def display_results(results: List[Dict[str, Any]], show_similarity: bool = True):
    """Pretty print search results.
    
    Args:
        results: List of search results
        show_similarity: Whether to display similarity scores
    """
    if not results:
        print("No results found.")
        return
        
    print(f"\n=== Found {len(results)} results ===\n")
    
    for i, result in enumerate(results, 1):
        print(f"Result #{i}:")
        print(f"URL: {result['url']}")
        print(f"Page ID: {result['page_id']}")
        print(f"Segment ID: {result['segment_id']}")
        
        if show_similarity and 'similarity' in result:
            print(f"Similarity: {result['similarity']:.4f}")
        if 'text_rank' in result:
            print(f"Text Rank: {result['text_rank']:.4f}")
        if 'combined_score' in result:
            print(f"Combined Score: {result['combined_score']:.4f}")
            
        text = result['page_segment']
        if len(text) > 300:
            text = text[:297] + "..."
            
        print(f"Content: {text}")
        print("-" * 80)

if __name__ == "__main__":
    print("Starting the vector database query tool...")


Starting the vector database query tool...


In [43]:
from vector_processor import VectorProcessor

vector_processor = VectorProcessor()
querier = VectorDBQuerier(vector_processor)

Connecting to the database VectorDB01


In [44]:
def display_results(results: List[Dict[str, Any]], show_similarity: bool = True):
    """Pretty print search results.
    
    Args:
        results: List of search results
        show_similarity: Whether to display similarity scores
    """
    if not results:
        print("No results found.")
        return
        
    print(f"\n=== Found {len(results)} results ===\n")
    
    for i, result in enumerate(results, 1):
        print(f"Result #{i}:")
        print(f"URL: {result['url']}")
        print(f"Page ID: {result['page_id']}")
        print(f"Segment ID: {result['segment_id']}")
        
        if show_similarity and 'similarity' in result:
            print(f"Similarity: {result['similarity']:.4f}")
        if 'text_rank' in result:
            print(f"Text Rank: {result['text_rank']:.4f}")
        if 'combined_score' in result:
            print(f"Combined Score: {result['combined_score']:.4f}")
            
        text = result['page_segment']
        if len(text) > 300:
            text = text[:297] + "..."
            
        print(f"Content: {text}")
        print("-" * 80)


def run_search(query=None, url=None, page=None, limit=5, threshold=0.5, hybrid=False):
    vector_processor = VectorProcessor()
    querier = VectorDBQuerier(vector_processor)

    try:
        if page is not None:
            url_result, content = querier.get_page_content(page)
            if url_result and content:
                print(f"\n=== Content for page {page} ===")
                print(f"URL: {url_result}")
                print("\nContent:")
                print("=" * 80)
                print(content)
                print("=" * 80)
            else:
                print(f"No page found with ID {page}")
                
        elif query and url:
            print(f"Searching for '{query}' within URLs matching '{url}'...")
            results = querier.url_content_search(url, query, limit)
            display_results(results)
            
        elif url:
            print(f"Searching for content in URLs matching '{url}'...")
            results = querier.url_content_search(url, limit=limit)
            display_results(results, show_similarity=False)
            
        elif hybrid and query:
            print(f"Performing hybrid search for '{query}'...")
            results = querier.keyword_and_semantic_search(query, limit)
            display_results(results)
            
        elif query:
            print(f"Searching for '{query}'...")
            results = querier.semantic_search(query, limit, threshold)
            display_results(results)
            
        else:
            print("No valid input parameters provided.")
    
    except Exception as e:
        print("An error occurred:", e)

**EXAMPLE USAGE**

1. Semantic search only

In [45]:
run_search(query="erasmus+")


Connecting to the database VectorDB01
Searching for 'erasmus+'...

=== Found 5 results ===

Result #1:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razpisi/Erasmus+ 2023_2024/Seznam_BA_za_ERASMUS_.pdf
Page ID: 1608
Segment ID: 57931
Similarity: 0.6158
Content: Erasmus+
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/upload/Izmenjave/Razpis Erasmus 2018_2019/Seznam_BA_za_ERASMUS_za_2018_20.pdf
Page ID: 1606
Segment ID: 57459
Similarity: 0.6158
Content: Erasmus+
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Dokumenti strategija, ECHE listina, EPS/EPS_slo_MS_čistopis_2_eng_GB.pdf
Page ID: 1600
Segment ID: 56043
Similarity: 0.6158
Content: Erasmus+
--------------------------------------------------------------------------------
Result #4:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Ra

2. Semantic search with threshold and limit

In [27]:
run_search(query="erasmus +", limit=3, threshold=0.6)


Connecting to the database VectorDB01
Searching for 'erasmus +'...

=== Found 3 results ===

Result #1:
URL: https://www.fri.uni-lj.si/upload/Izmenjave/Razpis Erasmus 2018_2019/Seznam_BA_za_ERASMUS_za_2018_20.pdf
Page ID: 1606
Segment ID: 57459
Similarity: 0.6158
Content: Erasmus+
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razpisi/Erasmus+ 2022_2023/Seznam_BA_za_ERASMUS_.pdf
Page ID: 1607
Segment ID: 57695
Similarity: 0.6158
Content: Erasmus+
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/sl/o-izmenjavah#6
Page ID: 1598
Segment ID: 55571
Similarity: 0.6158
Content: Erasmus+
--------------------------------------------------------------------------------


3. Hybrid search

In [28]:
run_search(query="student exchange", hybrid=True)


Connecting to the database VectorDB01
Performing hybrid search for 'student exchange'...

=== Found 5 results ===

Result #1:
URL: https://www.fri.uni-lj.si/en/study-programmes
Page ID: 1441
Segment ID: 53956
Text Rank: 0.0333
Combined Score: 0.4624
Content: students on international exchange
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/mednarodne-izmenjave
Page ID: 3347
Segment ID: 115362
Text Rank: 0.0000
Combined Score: 0.4613
Content: študijske izmenjave
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/upload/Izmenjave/PROGRAM_ERASMUS_SMS_SMP.pdf
Page ID: 1461
Segment ID: 54379
Text Rank: 0.0000
Combined Score: 0.4613
Content: študijske izmenjave
--------------------------------------------------------------------------------
Result #4:
URL: https://www.fri.uni-lj.si/international-study-exchanges
Page ID: 1460
Segment ID: 54336
Text

4. Search with URL pattern and query (filtered search)

In [46]:
run_search(url="izmenjave", query="korea")


Connecting to the database VectorDB01
Searching for 'korea' within URLs matching 'izmenjave'...

=== Found 5 results ===

Result #1:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razpisi/Erasmus+ 2022_2023/Seznam_BA_za_ERASMUS_.pdf
Page ID: 1607
Segment ID: 57505
Similarity: 0.4850
Content: kazalo
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razpisi/13, 14 regija in 3 SVET/Seznam_BA_za_13_14_regijo_in_3_S - Copy 1.pdf
Page ID: 1604
Segment ID: 56797
Similarity: 0.4850
Content: kazalo
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Dokumenti strategija, ECHE listina, EPS/EPS_slo_MS_čistopis_2_eng_GB.pdf
Page ID: 1600
Segment ID: 55853
Similarity: 0.4850
Content: kazalo
--------------------------------------------------------------------------------
Result #4:
URL: http

5. Search with URL pattern only

In [ ]:
run_search(url="erasmus")

Connecting to the database VectorDB01
Searching for content in URLs matching 'erasmus'...

=== Found 5 results ===

Result #1:
URL: https://www.fri.uni-lj.si/en/news/article/erasmus-study-abroad-fair
Page ID: 690
Segment ID: 36734
Content: Home > News
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/en/news/article/erasmus-study-abroad-fair
Page ID: 690
Segment ID: 36735
Content: News Published 7. december 2022 Erasmus+ Study Abroad Fair Other notifications You are invited to the Erasmus+ Study Abroad Fair, which will take place on Wednesday, 14 December 2022, from 11 am to 2 pm in the FRI Lobby. At 11:00 in the FRI Graduation Room, we will be presenting the call for Eras...
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/en/news/article/erasmus-study-abroad-fair
Page ID: 690
Segment ID: 36736
Content: News Published 7. december 2022
-----

6. Fetch content by page ID

In [32]:
run_search(page=666)

Connecting to the database VectorDB01

=== Content for page 666 ===
URL: https://www.fri.uni-lj.si/en/employees/janez-demsar

Content:
Home > About > Employees >
<<<PARAGRAPH>>>
Staff Links SICRIS Prof. dr. Janez Demšar Full Professor E: Send a message Your email * Your message * Office hours: Anytime (you may want to send me a mail before) Room: R3.18 - Kabinet Description I teach at the Faculty of Computer and Information Science (FRI) and the Faculty of Education at the University of Ljubljana (UL). In the past, I also taught at the Higher School of Economics in Moscow and Baylor College of Medicine in Houston. My research focuses on machine learning, visualization, and I know a bit of statistics. As a programmer, I contribute to the development of Orange , the largest open-source system for data analytics, as well as other smaller projects such as Trubar and Baycomp . One of the more fun ones—which actually belongs in the next paragraph—is Bobrova pustolovščina ( The Beaver’s Adven

In [20]:
run_search(query="erazmus+", hybrid=True, limit=5)


Connecting to the database VectorDB01
Performing hybrid search for 'erazmus+'...

=== Found 5 results ===

Result #1:
URL: https://www.fri.uni-lj.si/sl/o-izmenjavah#6
Page ID: 1598
Segment ID: 55571
Text Rank: 0.0000
Combined Score: 0.3897
Content: Erasmus+
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/upload/Izmenjave/Razpis Erasmus 2018_2019/Seznam_BA_za_ERASMUS_za_2018_20.pdf
Page ID: 1606
Segment ID: 57459
Text Rank: 0.0000
Combined Score: 0.3897
Content: Erasmus+
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razpisi/Erasmus+ 2022_2023/Seznam_BA_za_ERASMUS_.pdf
Page ID: 1607
Segment ID: 57695
Text Rank: 0.0000
Combined Score: 0.3897
Content: Erasmus+
--------------------------------------------------------------------------------
Result #4:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razp

In [21]:
run_search(query="erazmus+", hybrid=False, limit=5)

Connecting to the database VectorDB01
Searching for 'erazmus+'...

=== Found 5 results ===

Result #1:
URL: https://www.fri.uni-lj.si/upload/mednarodne_izmenjave/Razpisi/13, 14 regija in 3 SVET/Seznam_BA_za_13_14_regijo_in_3_S - Copy 1.pdf
Page ID: 1604
Segment ID: 56987
Similarity: 0.6494
Content: Erasmus+
--------------------------------------------------------------------------------
Result #2:
URL: https://www.fri.uni-lj.si/upload/Izmenjave/Diploma izmenjava/GOINGLOBAL_predstavitev_z_dodani.pdf
Page ID: 1602
Segment ID: 56515
Similarity: 0.6494
Content: Erasmus+
--------------------------------------------------------------------------------
Result #3:
URL: https://www.fri.uni-lj.si/upload/Obrazec_Mentor_zakljucna_naloga_ - Copy 1.pdf
Page ID: 1601
Segment ID: 56279
Similarity: 0.6494
Content: Erasmus+
--------------------------------------------------------------------------------
Result #4:
URL: https://www.fri.uni-lj.si/upload/Izmenjave/Ocene.pdf
Page ID: 1603
Segment ID: 56751
